# Working with Two Tables

In [1]:
%load_ext sql

# Windows users need to specify password
%sql postgres://isdb16@localhost/postgres

'Connected: isdb16@postgres'

In [2]:
%%sql 

DROP TABLE if EXISTS Students CASCADE ;

CREATE TABLE Students (
    PRIMARY KEY(sid),
    sid   text,
    name  text,
    gpa   NUMERIC(3,2)
);

INSERT INTO Students (sid, name, gpa)
     VALUES
     ('s1', 'Mary', 4),
     ('s2', 'Jack', 3),
     ('s3', 'Pat', 2);
        
  SELECT * 
    FROM Students
ORDER BY sid;

Done.
Done.
3 rows affected.
3 rows affected.


sid,name,gpa
s1,Mary,4.00
s2,Jack,3.00
s3,Pat,2.00


In [3]:
%%sql

DROP TABLE if EXISTS Hobbies CASCADE ;

CREATE TABLE Hobbies(
  sid  text,
  name text
);

INSERT INTO Hobbies
     VALUES
     ('s1', 'Running'),
     ('s1', 'Swimming'),
     ('s2', 'Swimming'),
     ('s2', 'Squash'),
     ('s2', 'Tennis') ;
        
  SELECT * 
    FROM Hobbies
ORDER BY sid;

Done.
Done.
5 rows affected.
5 rows affected.


sid,name
s1,Running
s1,Swimming
s2,Swimming
s2,Squash
s2,Tennis


**`SELECT` all fields from TWO tables at the same time**

In [4]:
%%sql
--solution

  SELECT * -- Students.name, Hobbies.name
    FROM Students, Hobbies
ORDER BY Students.sid;

15 rows affected.


sid,name,gpa,sid_1,name_1
s1,Mary,4.00,s1,Running
s1,Mary,4.00,s1,Swimming
s1,Mary,4.00,s2,Swimming
s1,Mary,4.00,s2,Squash
s1,Mary,4.00,s2,Tennis
s2,Jack,3.00,s1,Running
s2,Jack,3.00,s1,Swimming
s2,Jack,3.00,s2,Swimming
s2,Jack,3.00,s2,Squash
s2,Jack,3.00,s2,Tennis


This is termed the **cross product** of two tables (relations) which each entry in one table is matched (joined) with **ALL** entries of the other table.  Note that if the two tables being joined have the same attribute name then Postgres renames them for us for differentiation.

The cross product of two tables is often not that useful.  Rather we want to align corresponding rows i.e., join rows together that have information about the same entity.

One way of doing this is:

In [5]:
%%sql
--code

  SELECT * 
    FROM Students, Hobbies
   WHERE Students.sid = Hobbies.sid
ORDER BY Students.sid;

5 rows affected.


sid,name,gpa,sid_1,name_1
s1,Mary,4.00,s1,Running
s1,Mary,4.00,s1,Swimming
s2,Jack,3.00,s2,Swimming
s2,Jack,3.00,s2,Squash
s2,Jack,3.00,s2,Tennis


The above way of joining two tables using a `WHERE` clause is **NOT** the recommended way.  A `WHERE` clause should _only_ be used for _filtering_ rows, based on a condition, as they are read in.

To join two tables together, use the recommended `JOIN` clause:

In [6]:
%%sql

  SELECT *
    FROM Students as s
         JOIN Hobbies as h
         ON s.sid = h.sid
ORDER BY s.sid;

5 rows affected.


sid,name,gpa,sid_1,name_1
s1,Mary,4.00,s1,Running
s1,Mary,4.00,s1,Swimming
s2,Jack,3.00,s2,Swimming
s2,Jack,3.00,s2,Squash
s2,Jack,3.00,s2,Tennis


**How many hobbies is each student doing?**

In [7]:
%%sql

  SELECT  s.name, count(s.name)
    FROM  Students as s
          JOIN Hobbies as h
          ON s.sid = h.sid
GROUP BY  s.name
ORDER BY  s.name;

2 rows affected.


name,count
Jack,3
Mary,2


**Hmmm ... Pat is not doing any hobbies and we would like to know that 
information. How can we get that?**

We need to use a particular form of nested query termed a _correlated nested query_. This essentially amount to a nested loop.

The outer most `SELECT` statement iterates over each row of `Students`.  For each row of `Students` the inner `SELECT` iterates over `Hobbies`.   

In [8]:
%%sql 

SELECT  s.sid, s.name, 
         (SELECT count(h.sid) 
            FROM Hobbies as h 
           WHERE h.sid=s.sid) as "# of hobbies"
  FROM Students as s;

3 rows affected.


sid,name,# of hobbies
s1,Mary,2
s2,Jack,3
s3,Pat,0


### More details on how correlated nested queries work

SQL is a declarative language (you say **WHAT** to do) whereas most languages you may have seen already (Java, Python etc) are procedural languages (you say **HOW** to do it).

The following analogy may help.

Think of a row in a SQL table as a Python _dictionary_ and column names as _keys_ to that dictionary.  So if `s` is a single row of the `Students` table then we can access the individual cells by using the column names as keys e.g., `s['sid']`, `s['name']`, and `s['gpa']`.

A SQL table can be thought of as a list of rows and hence a list of dictionaries.  Hence a SQL table in Python terms could be viewed as having the following structure:

```
[ {...        },
  {...        },
  {...        },
  ... ]
 ```
 
As we discussed in class, a `FROM` statement is equivalent to a `FOR` statement in a procedural language. So,

```
SELECT s.sid, s.name
  FROM Students as s 
 ```
 
is equivalent to:

```
for s in Students:
    print( s['sid'], s['name'] )
```

What we want is: for a given `sid`, count the number of times it occurs in the `Hobbies` table. Expanding this further we get:
 
   1. Given a Student, `s`, count the number of times `s['sid']` occurs in       the `Hobbies` table 
   2. Do this for all `Students`

Step 1, procedurally is:

```
def count(Hobbies, s):
    for h in Hobbies:
       cnt = 0
       if h['sid'] == s['sid']:
          cnt += 1
    return cnt
```

The declarative version in SQL is:
 
``` 
SELECT count(h.sid) 
  FROM Hobbies AS h 
 WHERE h.sid = s.sid
```

Step 2, procedually in Python is

```
for s in Students:
   cnt = count( Hobbies, s)
   print( s['sid'], s['name'], cnt)
```

declaratively in SQL we have

```
SELECT s.sid, s.name, <step-1>
  FROM Students AS s;
```

The above `s` is accessed non-locally in step-1, similar to how non-local variables are accessed in Python.
 
_What takes many lines of code in Java, can be said in a few lines of Python.  The joy of Python!  What can be said in many lines of (procedural) Python, can be said in a few lines of SQL.  But, of course, Python is general purpose whereas SQL has been crafted for working specifically with tables._

In [9]:
%%sql

select avg(gpa)
from Students;

1 rows affected.


avg
3.0000000000000000


In [10]:
%%sql 

select name, gpa
from students
where gpa >= some(select 3.5)

1 rows affected.


name,gpa
Mary,4.00
